# Inner Visualizer Process

---

## 1. Set API Keys

In [ ]:
import os

# Load API Keys
from config.secret_keys import OPENAI_API_KEY

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## 2. Import Modules

In [1]:
# directory setting
from pathlib import Path

# Define State
from typing import Annotated, List
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
import operator

# Define Tools
import pandas as pd
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

## 3. Directory Setting

In [ ]:
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / 'data' # 데이터 저장용 디렉토리
CHART_DIR = BASE_DIR / 'charts' # 차트 저장용 디엑토리

## 4. Build Application

## 4-1. Define State

In [ ]:
class AnalysisState(TypedDict):
    company: str
    messages: Annotated[List[BaseMessage], operator.add]
    analyses: Annotated[List[dict], operator.add]
    combined_report: str

## 4-2. Define Tools

In [ ]:
@tool
def inner_visualizer(command:str):
    """
    The tool creates charts using create_pandas_dataframe_agent and stores them in the /charts folder.
    """

    inner_data = pd.read_csv(DATA_DIR / "stock_data.csv")
    custom_prefix = """
        Please make the chart and save in './charts' folder.
        data path is './data/stock_data.csv'
    """

    agent = create_pandas_dataframe_agent(
        ChatOpenAI(model="gpt-4o"),
        [inner_data],
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        allow_dangerous_code=True,
        prefix = custom_prefix
    )

    result = agent.run(command)

    return result

### 4-3. Define Agent